In [1]:
import os
import glob
import json


from wand.image import Image as wImg
import pandas as pd
import numpy as np

import torch
from torchvision.transforms import v2
from PIL import Image


import torchvision.models as models


from torch.utils.data import DataLoader

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Torch device: {device}")

Torch device: cuda


Objetivo desse programa é:


função: ler_svg
-------------------------------------------------------------------------------------------------------------------------------
ler arquivos .svg, salvar o nome e o path para cada arquivo em um data frame

nome do dataframe vai ser: icons-data-base
coluna dos nome dos arquivos .svg: Nome
coluna com os paths dos arquivos .svg: svg path

-------------------------------------------------------------------------------------------------------------------------------

função: converter_svg_para_png
-------------------------------------------------------------------------------------------------------------------------------
transformar os arquivos .svg em .png

através da coluna: svg path, abrirei os arquivos e converterei em .png e salvarei em uma nova pasta, chamada icons-png-database
o nome de cada arquivo .png será escrito de acordo com a coluna: Nome

salvarei o path para os arquivos .png em uma coluna do data frame chamada: png path

-------------------------------------------------------------------------------------------------------------------------------

função: converter_png_em_pil_image
-------------------------------------------------------------------------------------------------------------------------------
converterei os arquivos .png em PIL Image

a função vai ler o path para os arquivos .png através da coluna: png path

salvarei os arquivos PIL Image em uma lista chamada: icons_pil_images

-------------------------------------------------------------------------------------------------------------------------------

função: converter_pil_em_tensor
-------------------------------------------------------------------------------------------------------------------------------

converterei as PIL Image em torch.tensor

a função para converter irá converter as images da lista icons_pil_images

o tensor de cada imagem será salvo em uma lista para ser usada na função analisar_tensor

-------------------------------------------------------------------------------------------------------------------------------

função: analisar_tensor
-------------------------------------------------------------------------------------------------------------------------------

Irei analisar cada tensor para extrair as caracteristicas de cada imagem

os tensores serão dados atraves da coluna do data frame: Tensor

a função ira retornar as caracteristicas de cada imagem em forma de arrays, 
esses arrays serão salvos em uma nova coluna do data frame chamada: Caracteristicas Array

-------------------------------------------------------------------------------------------------------------------------------

tenho que ser capaz de salvar os arrays em um arquivo para depois usa-los em um outro programa

In [3]:
def ler_svg(path_to_svgs):

    path_svgs = glob.glob(os.path.join(path_to_svgs, '*.svg'))

    nome_svgs = list()
    
    for path in path_svgs:

        nome_n_formatado = path.rsplit(("\\"))[-1]
        nome_formatado = nome_n_formatado.replace(".svg" , "")
        nome_svgs.append(nome_formatado)

    icons_data_base = pd.DataFrame({ 'Nome': nome_svgs, 'svg path': path_svgs})

    return icons_data_base

In [4]:
def converter_svg_para_png(data_base, pasta_dos_pngs='icons-png-database'):
    
    if not os.path.exists(pasta_dos_pngs):
        os.makedirs(pasta_dos_pngs)

    png_paths = list()

    for i, row in data_base.iterrows():
        svg_path = row['svg path']
        nome = row['Nome']

        nome_arq_png = f"{nome}.png"
        png_path = os.path.join(pasta_dos_pngs, nome_arq_png)

        with wImg(filename = svg_path) as img:
            img.format = 'png'
            img.save(filename = png_path)
            png_paths.append(png_path)

    data_base['png path'] = png_paths

    return data_base

In [5]:
def converter_png_em_pil_image(data_base, pasta_dos_pngs='icons-png-database'):
    
    icons_pil_image = list()
    
    for filename in os.listdir(pasta_dos_pngs):
        path = os.path.join(pasta_dos_pngs, filename)
        icon = Image.open(path).convert('RGB')

        icons_pil_image.append(icon)


    data_base['PIL Image'] = icons_pil_image
    
    return data_base


In [6]:
def converter_pil_em_tensor(data_base):
    
    lista_tensores = list()

    img = v2.Compose([
                    v2.Resize((224, 224)),
                    v2.ToImage(), 
                    v2.ToDtype(torch.float32, scale=True),
                    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    
    for pil_image in data_base['PIL Image']:

        img_tensor = img(pil_image)
   
        lista_tensores.append(img_tensor)
   

    return lista_tensores


In [7]:
def analisar_tensores(data_base, lista_tensores, batch_size = 20):

    torch.cuda.empty_cache()

    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    model = torch.nn.Sequential(*(list(model.children())[:-1]))

    model.to(device).eval()



    dataloader = DataLoader(lista_tensores, batch_size = batch_size, shuffle=False)
    
    array_caracteristicas = list()

    for batch in dataloader:
        imgs = batch.to(device) 
        if imgs.dim() == 3: 
            imgs = imgs.unsqueeze(0)

        with torch.no_grad():

            features = model(imgs).squeeze() 
            array_caracteristicas.extend(features.cpu().numpy())

            
    data_base['Array características'] = array_caracteristicas[:len(data_base)]

    data_base['Array características'] = data_base['Array características'].apply(lambda x: json.dumps(x.tolist()))

    return data_base

In [8]:
def salvar_dataframe(data_base, nome_arquivo_hdf5):
    data_base.to_hdf(nome_arquivo_hdf5, key="data", mode="w")

In [9]:
def ler_dataframe(nome_arquivo_hdf5="icons_data_base.h5"):

    nome_data_base = pd.read_hdf(nome_arquivo_hdf5, key="data")
    nome_data_base['Array características'] = nome_data_base['Array características'].apply(lambda x: np.array(json.loads(x)))
    
    return nome_data_base

In [10]:
def main(path_to_svgs, pasta_dos_pngs = 'icons-png-database', nome_arquivo_hdf5 = 'icons-data-frame.h5', batch_size = 20):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Torch device: {device}")

    data_base = ler_svg(path_to_svgs)

    data_base = converter_svg_para_png(data_base, pasta_dos_pngs)

    data_base = converter_png_em_pil_image(data_base, pasta_dos_pngs)

    lista_tensores = converter_pil_em_tensor(data_base)

    data_base = analisar_tensores(data_base, lista_tensores, batch_size)

    salvar_dataframe(data_base, nome_arquivo_hdf5)

    print(f"\nPasta {pasta_dos_pngs} criada com sucesso\n")
    print(f'Arquivo .hdf5 chamado {nome_arquivo_hdf5} criado com sucesso\n')
    print('-----------------------------------------------------------------------------------------------------------------------------------------------------------\n')


In [11]:
main(r'material-design-icons\svg\filled')

Torch device: cuda

Pasta icons-png-database criada com sucesso

Arquivo .hdf5 chamado icons-data-frame.h5 criado com sucesso

-----------------------------------------------------------------------------------------------------------------------------------------------------------



C:\Users\IgorG\AppData\Local\Temp\ipykernel_3924\1760923310.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Nome', 'svg path', 'png path', 'PIL Image', 'Array características'], dtype='object')]

  data_base.to_hdf(nome_arquivo_hdf5, key="data", mode="w")
